In [1]:
import argparse, os
import torch
import numpy as np

import os, h5py, json
from tqdm import tqdm
from PIL import Image

from skimage import transform, io, img_as_float
import torchvision.transforms as transforms
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
from Transparency.Trainers.DatasetQA import *
from Transparency.ExperimentsQA import *
from Transparency.common_code.common import *
from Transparency.Trainers.PlottingQA import generate_graphs
from Transparency.configurations import configurations_qa
from Transparency.Trainers.TrainerQA import Trainer, Evaluator

task = "qa"
torch.backends.cudnn.enabled = False

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str)
parser.add_argument("--data_dir", type=str)
parser.add_argument("--output_dir", type=str)
parser.add_argument('--encoder', type=str, choices=['cnn', 'lstm', 'average', 'all'])
parser.add_argument('--attention', type=str, choices=['tanh', 'dot', 'all'])
parser.add_argument("--gpu", type=str, default='2')
args, extras = parser.parse_known_args(args=[])
args.extras = extras


args.gpu = '0'
args.output_dir = 'outputs/qa'
args.data_dir = './'

args.batch_size = 256


os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

In [38]:
############################ Generate Results for Table Begin ################################## 

all_dataset_names = ['qqp', 'babi_1', 'babi_2', 'babi_3', 'snli']
all_encoder_names = ['cnn', 'lstm']
all_attention_names = ['dot', 'tanh']
all_exp_names = ["rand", "ig", "att", "att*grad", "att_norm", \
    "att*grad_abs", "att*grad_only_grad", "att*grad_sign", "grad_cam", "input*grad", ]
all_exp_metric_names = ["AUCTP", "Violation", "Sufficiency", "Comprehensiveness", "RC"]
all_replace_names = ["slice_out", "zeros_mask", "att_mask"]


# ["att*grad", "att", "ig", "rand", "input*grad", "att_norm"]
dataset_names = ['babi_1']
encoder_names = ['lstm']
attention_names = ['tanh']
exp_names = ["rand", "att"]
# exp_names = ["ig", "rand", "att", "att*grad", "input*grad", "att_norm"]

exp_metric_names = ["Violation"]
replace_names = ["slice_out", "zeros_mask", "att_mask"]
args.batch_size = 256


total_scores_dict = {dataset_name:{} for dataset_name in dataset_names}
for dataset_name in dataset_names: # Dataset Name
    if dataset_name not in total_scores_dict:
        total_scores_dict[dataset_name] = {}
    dataset_dict = total_scores_dict[dataset_name]

    # load dataset
    args.dataset = dataset_name
    dataset = datasets[args.dataset](args)
    if args.output_dir is not None :
        dataset.output_dir = args.output_dir
    dataset.display_stats()

    # mask size setup
    dataset_len = len(dataset.test_data.A)
    minlen = len(dataset.test_data.Q[0])
    mask_size = 1
    group_size = 20
    if minlen > group_size:
        # in case of very long sequence, we use mask_size to reduce iteration times
        mask_size += (minlen//group_size)

    for encoder_name in encoder_names: # Model Type
        args.encoder = encoder_name
        for attention_type in attention_names: # tanh  # Attention Type
            args.attention = attention_type
            if args.attention in ['dot'] :
                e = args.encoder + '_dot'
            else:
                e = args.encoder

            config = e
            if e not in dataset_dict:
                dataset_dict[e] = {}
            
            config = configurations_qa[config](dataset)
            latest_model_path = get_latest_model(os.path.join(config['training']['basepath'], config['training']['exp_dirname']))
            evaluator = Evaluator(dataset, latest_model_path, _type=dataset.trainer_type)
            evaluator.model.bsize = args.batch_size
            # predictions, attentions, metrics = evaluator.evaluate(dataset.test_data, save_results=False, return_metrics=True)

            for exp_type in exp_names: # Explanation Type
                load_cached_exp = False
                print("#"*20, "dataset:{}, model:{}, exp_type:{} ".format(dataset_name, encoder_name+"_"+attention_type, exp_type), "#"*20)
                if exp_type not in dataset_dict[e]:
                    scores = {n:[] for n in all_exp_metric_names}
                else:
                    scores = dataset_dict[e][exp_type]
                    for exp_metric in exp_metric_names:
                        scores[exp_metric] = []

                for i, exp_metric in enumerate(exp_metric_names):
                    for delta_type in replace_names: # Replacement methods
                        print("replace_func:{}, metric:{}".format(delta_type, exp_metric))
                        torch.cuda.empty_cache()
                        outputs = evaluator.replace_and_run_experiment(e, dataset.test_data, exp=exp_type, \
                            delta_type=delta_type, exp_metric=exp_metric, mask_size=mask_size, plot=True, \
                            load_exp_scores=load_cached_exp, load_perturbation_scores=False)

                        for calculated_metric in outputs[-1]:
                            scores[calculated_metric].append(outputs[-1][calculated_metric])
                        
                        load_cached_exp = True

                dataset_dict[e][exp_type] = scores
                mean_scores = {n: sum(scores[n])/3 for n in scores}
                print(mean_scores)
                print(scores)


/home/lyb/.conda/envs/python3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
INFO - 2022-01-20 11:31:22,584 - type = cnn
INFO - 2022-01-20 11:31:22,585 - vocab_size = 24
INFO - 2022-01-20 11:31:22,586 - embed_size = 50
INFO - 2022-01-20 11:31:22,587 - hidden_size = 8
INFO - 2022-01-20 11:31:22,588 - kernel_sizes = [1, 3, 5, 7]
INFO - 2022-01-20 11:31:22,588 - activation = relu
INFO - 2022-01-20 11:31:22,589 - type = relu
INFO - 2022-01-20 11:31:22,590 - type = relu
INFO - 2022-01-20 11:31:22,591 - pre_embed = None
INFO - 2022-01-20 11:31:22,594 - type = cnn
INFO - 2022-01-20 11:31:22,595 - vocab_size = 24
INFO - 2022-01-20 11:31:22,596 - embed_size = 50
INFO - 2022-01-20 11:31:22,597 - hidden_size = 8
INFO - 2022-01-20 11:31:22,597 - kernel_sizes = [1, 3, 5, 7]
INFO - 2022-01-20 11:31:22,598

{'vocab_size': 24, 'embed_size': 50, 'train_size': [('Overall', 8500)], 'test_size': [('Overall', 1000)], 'min_length': 11, 'max_length': 67, 'mean_length': 36.30764705882353, 'std_length': 17.60362430243599}
#################### dataset:babi_1, model:cnn_dot, exp_type:att*grad  ####################
replace_func:slice_out, metric:Violation
/home/lyb/Transparency/data/qa/cnn_dot/att*grad
Violation mask size must be 1


100%|██████████| 4/4 [00:02<00:00,  1.65it/s]


save Violation_slice_out.pkl perturbation scores to /home/lyb/Transparency/data/qa/cnn_dot/att*grad
num. of violators: 142, ratio:14.20%
replace_func:zeros_mask, metric:Violation
/home/lyb/Transparency/data/qa/cnn_dot/att*grad
load exp scores from outputs/qa/babi_1/cnn(1,3,5,7)+dot/Mon_Jan_17_22:19:26_2022/att*grad_bz256.pkl
Violation mask size must be 1


100%|██████████| 4/4 [00:02<00:00,  1.74it/s]


...

save Sufficiency_slice_out.pkl perturbation scores to /home/lyb/Transparency/data/qa/lstm/att_norm
Calculate Sufficiency for [0.5  0.8  0.9  0.95]
replace_func:zeros_mask, metric:Sufficiency
/home/lyb/Transparency/data/qa/lstm/att_norm
load exp scores from outputs/qa/babi_1/lstm+tanh/Mon_Jan_17_22:12:21_2022/att_norm_bz256.pkl


100%|██████████| 4/4 [00:00<00:00, 10.99it/s]


save Sufficiency_zeros_mask.pkl perturbation scores to /home/lyb/Transparency/data/qa/lstm/att_norm
Calculate Sufficiency for [0.5  0.8  0.9  0.95]
replace_func:att_mask, metric:Sufficiency
/home/lyb/Transparency/data/qa/lstm/att_norm
load exp scores from outputs/qa/babi_1/lstm+tanh/Mon_Jan_17_22:12:21_2022/att_norm_bz256.pkl


100%|██████████| 4/4 [00:00<00:00, 10.88it/s]

save Sufficiency_att_mask.pkl perturbation scores to /home/lyb/Transparency/data/qa/lstm/att_norm
Calculate Sufficiency for [0.5  0.8  0.9  0.95]
{'AUCTP': 0.21132499999999999, 'Violation': 0.004666666655490796, 'Sufficiency': 0.3325740723163714, 'Comprehensiveness': 0.7937465707461039, 'RC': 0.4941190481185913}
{'AUCTP': [0.173925, 0.1759, 0.28415], 'Violation': [0.004999999888241291, 0.004999999888241291, 0.004000000189989805], 'Sufficiency': [0.6665859222412109, 0.33114752173423767, -1.1227026334381662e-05], 'Comprehensiveness': [0.8308385610580444, 0.829367995262146, 0.7210331559181213], 'RC': [0.35103297233581543, 0.4343900680541992, 0.6969341039657593]}


In [42]:
import pickle
import os

data_dir = "./data"
with open(os.path.join(data_dir, "QA_results.pkl"), 'wb') as f:
    pickle.dump(total_scores_dict, f)